
### Question 1
#### Question 1.(a) Explain how k-fold cross-validation is implemented

    1. We randomly divide the set of observations into k groups of approximately equal size.
    
    2. We'll train the model for k times. Every time we select one group as our validation set and the other k-1 groups as our training sets.
    
    3. Then, we have k estimates of the test error.
    
    4. Finally, we average these values to get the k-fold CV estimate.

#### Question 1.(b) What are the advantages and disadvantages of k-fold cross validation relative to
    1. The validation set approach; 
    Advantage of k-fold CV: Because only a relatively smaller part of observations are used in the model training, the model might perform worse, which can result in the overestimation of the test error. Since the training-testing datasets are randomly divided, the validation estimate of the test error may be highly variable. 
    
    Advantages of the validation set approach: The validation set approach is faster than k-fold CV for that we only need to fit the model once and calculate the error once.
    
    2. LOOCV
    Advantages of k-fold CV: LOOCV requires fitting model n times, which can be computationally expensive, whereas k-fold CV can be applied to almost any models for its feasibility. Moreover, k-fold CV often gives more accurate estimates of the test error rate than LOOCV.
    
    Advantages of LOOCV:  Because of the bias-variance trade-off, LOOCV may give approximately unbiased (the result doesn't change too much) estimates of the test error, even though LOOCV has higher variance than k-fold CV.

<font size="3">
</font>

   <font size="3"> 
</font>

In [1]:
import pandas as pd

redwine_df = pd.read_csv("redwine.csv")

redwine_df["final_quality"] = 0

redwine_df["final_quality"][redwine_df["quality"] > redwine_df["quality"].mean()] = 1

redwine_df["final_quality"].head()

X = redwine_df.iloc[:, :-2]
y = redwine_df.iloc[:, -1]

D:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [2]:
print(X.shape)
X.head()

(1599, 11)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


In [3]:
print(y.shape)
y.head()

(1599,)


0    0
1    0
2    0
3    1
4    0
Name: final_quality, dtype: int64

-----------------------------------------------



### Question 2

#### Question 2.(a) Fit a logistic regression model that uses all predictors (except quality) to predict final_quality.

In [4]:
from sklearn.linear_model import LogisticRegression

logit_model = LogisticRegression().fit(X,y)

D:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


#### Question 2. (b)Using the validation set approach, estimate the test error of this model. In order to do this,
you must perform the following steps:
    i. Split the sample set into a training set and a validation set.
    
    ii. Fit a multiple logistic regression model using only the training observations.
    
    iii. Obtain a prediction of final_quality for each wine in the validation set by computing   
    the posterior probability of high-quality for that wine, and classifying the wine to the
    high-quality category if the posterior probability is greater than 0.5.
    
    iv. Compute the validation set error, which is the fraction of the observations in the
    validation set that are misclassified.

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# i. Split the sample set into a training set and a validation set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=666)

# ii. Fit a multiple logistic regression model using only the training observations.
logit_model = LogisticRegression().fit(X_train, y_train)


# iii. Obtain a prediction of final_quality for each wine in the validation set.
y_pred_prob = logit_model.predict_proba(X_test)
# print(logit_model.classes_)
# print(y_pred_prob.shape)
y_pred_high_quality = y_pred_prob[:, 1]
y_pred_high_quality[y_pred_high_quality > 0.50] = 1
y_pred_high_quality[y_pred_high_quality < 0.50] = 0

# print(len(y_pred_high_quality))

# Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.
print("The validation set error is ", accuracy_score(y_test, y_pred_high_quality))


The validation set error is  0.725


D:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


### Question 3 
    a. Write a function, boot_fn(), that takes as input the redwine data set as well as an index of the
    observations, and that outputs the coefficient estimates for each predictor in the multiple
    logistic regression model.
    
    b. Use the boot() function together with your boot_fn() function to estimate the standard
    errors of the logistic regression coefficients for each predictor.
    
    c. Comment on the estimated standard errors obtained using the glm() function and using your
    bootstrap function. One way to get these values in python is use of statsmodel library.

In [6]:
import statsmodels.api as sm

X_add_constant  = sm.add_constant(X)
logit_model = sm.Logit(y, X_add_constant)
result = logit_model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.517706
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          final_quality   No. Observations:                 1599
Model:                          Logit   Df Residuals:                     1587
Method:                           MLE   Df Model:                           11
Date:                Mon, 18 Mar 2019   Pseudo R-squ.:                  0.2505
Time:                        18:44:26   Log-Likelihood:                -827.81
converged:                       True   LL-Null:                       -1104.5
                                        LLR p-value:                1.316e-111
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   42.9499     79.476      0.540      0.589    -112.820     198.720
fixed acidity            0.1360      0.098      1.381      0.167      -0.057       0.329
volatile acidity        -3.2817      0.488     -6.722      0.000      -4.239      -2.325
citric acid             -1.2743      0.563     -2.265      0.024      -2.377      -0.171
residual sugar           0.0553      0.054      1.029      0.304      -0.050       0.161
chlorides               -3.9157      1.569     -2.495      0.013      -6.992      -0.840
free sulfur dioxide      0.0222      0.008      2.698      0.007       0.006       0.038
total sulfur dioxide    -0.0164      0.003     -5.688      0.000      -0.022      -0.011
density                -50.9324     81.151     -0.628      0.530    -209.985     108.120
pH                      -0.3806      0.720     -0.528      0.597      -1.792       1.031
sulphates                2.7951      0.452      6.181      0.000       1.909       3.681
alcohol                  0.8668      0.104      8.319      0.000       0.663       1.071
========================================================================================
"""

In [23]:
from statsmodels.base.model import GenericLikelihoodModelResults

GenericLikelihoodModelResults.bootstrap(logit_model, nrep=100, method='nm', disp=0, store=1)

AttributeError: 'Logit' object has no attribute 'model'